<a href="https://colab.research.google.com/github/ApoorvaHegde21/AQI_Prediction/blob/main/Analysis_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This version of analysis and prediction uses LSTM to predict AQI indexes.


In [53]:
#Necessary imports
import pandas as pd
import numpy as np
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [57]:
#Links Google colab to Google Drive
from google.colab import drive
drive.mount('/content/drive')

#Read Data
data = pd.read_csv('/content/drive/MyDrive/final.csv')

#Extracts individual timestamp components
data['day'] = pd.to_datetime(data['Timestamp'],dayfirst=True).dt.day
data['month'] = pd.to_datetime(data['Timestamp'],dayfirst=True).dt.month
data['year'] = pd.to_datetime(data['Timestamp'],dayfirst=True).dt.year
data['hour'] = pd.to_datetime(data['Timestamp'],dayfirst=True).dt.hour


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
#Drop columns that have only zeroes and non numerical values.
data.drop(['Timestamp','Toluene (µg/m³)','RF (mm)','PM2.5 (µg/m³)','NH3 (µg/m³)'],axis=1,inplace=True)

#Determine the features and target variable.
X = data.copy()
y = X.pop('AQI')

train_X,val_X,train_y,val_y = train_test_split(X,y,test_size=0.2,random_state=1)
model = Sequential()
train_X = np.reshape(train_X.values, (train_X.shape[0], train_X.shape[1], 1))
val_X = np.reshape(val_X.values, (val_X.shape[0], val_X.shape[1], 1))

# LSTM layer of 25 units
model.add(LSTM(25, return_sequences=True, input_shape=(train_X.shape[1], 1)))
model.add(Dropout(0.3))

# Adds one more LSTM layer of 25 units
model.add(LSTM(25, return_sequences=False))
model.add(Dropout(0.3))

# Output layer
model.add(Dense(1,activation='linear'))

# Compiles the model
model.compile(optimizer=Adam(learning_rate = 0.2), loss='mean_squared_error')

# fits the model on training data
model.fit(train_X, train_y, epochs=10, batch_size=30, validation_data=(val_X, val_y))
predictions = model.predict(val_X)
#Error calculation
error = mean_absolute_error(val_y,predictions)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


30/30 ━━━━━━━━━━━━━━━━━━━━ 5s 46ms/step - loss: 778.8201 - val_loss: 226.7073
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 41ms/step - loss: 231.9254 - val_loss: 226.4935
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 41ms/step - loss: 232.4012 - val_loss: 226.0602
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 231.8741 - val_loss: 231.1542
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - loss: 235.4767 - val_loss: 223.5954
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 237.5189 - val_loss: 223.0135
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 226.1953 - val_loss: 226.8701
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 223.8318 - val_loss: 226.0765
Epoch 9/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - loss: 240.1031 - val_loss: 224.9703
Epoch 10/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 215.0565 - val_loss: 223.3376
7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step


In [59]:
print(error)

11.726063453987853
